In [ ]:
%reload_ext autoreload
%aimport -pandas, -numpy
%autoreload 2
import pandas as pd
import numpy as np
from dmyplant2 import (
    cred, MyPlant, Engine, FSMOperator, FSM, bokeh_show, get_cycle_data2, get_cycle_data, cvset, chart, FSMPlot_Start, dbokeh_chart, FSM_splot
)
from pprint import pprint as pp
import time


import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default='simple_white'
pio.renderers.default = "notebook"
    
cred()
mp = MyPlant(0)
Engine._list_cached_validations();

In [ ]:
pio.templates;

In [ ]:
fleet = mp.search_installed_fleet_by_contains_name("Forsa Hartmoor")
fleet = fleet.sort_values(by = "Engine ID",ascending=True).reset_index(drop='index')
motor = fleet.iloc[3]
modes = ['undefined','OFF','MAN','AUTO']
success = [True,False]
#success = [True]
fleet.T


In [ ]:
motor['IB Site Name'] + ' ' + motor['Engine ID'], modes, success

In [ ]:
t0 = time.time(); e=Engine.from_fleet(mp,motor);t1 = time.time()
fsm = FSMOperator(e, p_from=e['Commissioning Date'], p_to='2022-01-15', successtime=300); t2 = time.time()
fsm.run1(enforce=True, silent=False); t3 = time.time()
print(f"engine: {t1-t0:3.2f} sec\n   fsm: {t2-t1:3.2f} sec\n  run1: {t3-t2:3.2f} sec")

In [ ]:
rda = fsm.starts
rda['stime'] = rda.apply(lambda x: x['starttime'].strftime('%d.%m.%Y %H:%M:%S'), axis=1)
rda['etime'] = rda.apply(lambda x: x['endtime'].strftime('%d.%m.%Y %H:%M:%S'), axis=1)
rda[['no','success','mode','stime','etime','startpreparation','starter','speedup','idle','synchronize','loadramp','cumstarttime','targetoperation','rampdown','coolrun','runout']].round(2)

In [ ]:
from dowork import plot_now
dset = [{'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['dodgerblue','red'], 'unit':'kW'},
        {'col':['power_diff'],'ylim': [-300, 300], 'color':'black', 'unit':'kW/s'},
        {'col':['Various_Bits_CollAlarm'],'ylim': [0, 10], 'color':'black', 'unit':''},
        {'col':['Hyd_PressCrankCase'],'ylim': [-100, 100], 'color':'orange', 'unit':'mbar'},
        {'col':['Hyd_TempOilPumpOut'],'ylim': [0, 100], 'color':'violet', 'unit':'°C'},
        {'col':['Hyd_PressOilDif'],'ylim': [0, 10], 'color':'midnightblue', 'unit':'bar'},
        {'col':['Hyd_PressOil'],'ylim': [0, 10], 'color':'dodgerblue', 'unit':'bar'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue','unit':'rpm'}]
startversuch = rda.iloc[8];
data = get_cycle_data(fsm, startversuch, max_length=None, min_length=None, cycletime=1, silent=True, p_data=cvset(mp,dset))
data.head(5)

In [ ]:
data['power_diff'] = pd.Series(np.gradient(data['Power_PowerAct']))
#fig = FSM_splot(fsm, startversuch, data, dset, figsize=(16,5))
#fig = FSMPlot_Start(fsm, startversuch, data, v(mp,dset), dset, figsize=(16,10))
fig = plot_now(fsm, data, startversuch,cvset(mp,dset), dset, dfigsize=(16,10))
bokeh_show(fig)

In [ ]:
from dowork import plot_plotly
fig2 = plot_plotly(fsm, data, startversuch, cvset(mp,dset), dset, dfigsize=(16,10))
fig2.show(config={'scrollZoom': True})

In [ ]:
from dowork import plot_plotly
ddset = [{'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['dodgerblue','red'], 'unit':'kW'},
        {'col':['Hyd_TempOil'],'ylim': [0, 100], 'color':'violet', 'unit':'°C'},
        {'col':['Hyd_PressOil','Hyd_PressOilDif'],'ylim': [0, 10], 'color':['indigo','darkorange'], 'unit':'bar'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue','unit':'rpm'}]
dvset = cvset(mp,ddset)
data = get_cycle_data(fsm, startversuch, max_length=None, min_length=None, cycletime=1, silent=True, p_data=dvset)
#data['power_diff'] = pd.Series(np.gradient(data['Power_PowerAct']))
fig2 = plot_plotly(fsm, data, startversuch,dvset, ddset, dfigsize=(12,8))
config = dict({'scrollZoom': True})
fig2.add_vline(x=startversuch['endtime'],line_dash='solid',line_width=1,line_color='red')
fig2.add_hline(y=1500.0,yref='y2',ysizemode='scaled',line_dash='solid',line_color='blue',line_width=3)
fig2.show(config=config)
#[e for rec in ddset for e in rec['col']]

In [ ]:
# from dowork import plotly_verstehen
# plotly_verstehen(fsm, startversuch, data)

In [ ]:
#chart(data,dset, figsize=(16,10));

In [ ]:
svdetail=fsm.starts.iloc[6].to_dict()
#pp(svdetail);

In [ ]:
for i, v in enumerate(fsm.runlogdetail(startversuch, statechanges_only=True)):
    print(f"{i:3} {v}")

In [ ]:
print(fsm.results['runlogdetail'][0])
print(fsm.results['runlogdetail'][-1])

In [ ]:
print(len(fsm.results['runlogdetail']))